# ArangoDB NetworkX Adapter Getting Started Guide  

![arangodb](assets/logos/ArangoDB_logo.png) 

In this Notebook we learn how to export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database into [NetworkX](https://networkx.github.io/), the swiss army knife for graph analysis ion python.

# Setup

In [ ]:
!pip install -e /home/admin2/networkx-rework-test/adbnx_adapter

In [ ]:
from adbnx_adapter.

In [1]:
#from adbnx_adapter import ArangoDB_Networkx_Adapter

import oasis

# Create a Temporary ArangoDB Instance

In [2]:
# Request temporary instance from the managed ArangoDB Cloud Oasis.
con = oasis.getTempCredentials()

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

Reusing cached credentials.

https://5904e8d8a65f.arangodb.cloud:8529
Username: TUTcvzmdnahdqqtqg90jwhedl
Password: TUTy9qwp7aods0wa97zmyddl
Database: TUTlb2w846yqn2457bmox0lh


*Note: You can also use the above link and login data to explore the ArangoDB UI.*

TODO: arangorestore for the import of the imdb Fraud Detection data set

# Connect ArangoDB and NetworkX 

In [4]:
from adbnx_adapter.arangoDB_networkx_arango_adapter import ArangoDB_Networkx_Adapter
ma = ArangoDB_Networkx_Adapter(conn = con)
    

In [5]:
fraud_detection_attributes = { 'vertexCollections': {'account': {'Balance', 'account_type', 'customer_id', 'rank'},\
       'bank': {'Country', 'Id', 'bank_id', 'bank_name'},\
       'branch':{'City', 'Country', 'Id', 'bank_id', 'branch_id', 'branch_name'},\
       'Class':{'concrete', 'label', 'name'},\
       'customer': {'Name', 'Sex', 'Ssn', 'rank'}},\
                              'edgeCollections' : {'accountHolder': {'_from', '_to'},\
       'Relationship': {'_from', '_to', 'label', 'name', 'relationshipType'},\
       'transaction': {'_from', '_to'}}}

g = ma.create_networkx_graph(graph_name = 'FraudDetection',  graph_attributes =   fraud_detection_attributes)

AQLQueryExecuteError: [HTTP 404][ERR 1203] AQL: collection or view not found: account (while parsing)

In [ ]:
fraud_detection_attributes = { 'vertexCollections': {'Collection1': {'attribute1', 'attribute2'}}, 'edgeCollections' : {'Collection2' : {'att1','att2'}}

g1 = ma.create_networkx_graph(graph_name = 'FraudDetection', graph_attributes = fraud_detection_attributes)

In [ ]:
g.nodes()

In [ ]:
g.edges()